Task №1. Access settings

```
grant usage on schema public to planadmin;
grant usage on schema public to planmanager;

grant select on all tables in schema public to planadmin;
grant select on all tables in schema public to planmanager;

grant select, insert, update, delete on plan_data to planadmin;
grant select, insert, update, delete on plan_data to planmanager;

grant select, insert, update, delete on plan_status to planadmin;
grant select, update on plan_status to planmanager;

grant select, insert, update, delete on country_managers to planadmin;
grant select on country_managers to planmanager;

grant select, update on v_plan_edit to planmanager;
grant select on v_plan to planmanager;

create user ivan;

grant planadmin to ivan;

create user sophie;
create user kirill;

grant planmanager to sophie;
grant planmanager to kirill;

insert into country_managers (username, country) values 
('sophie', 'US'), ('sophie', 'CA'), ('kirill', 'FR'), ('kirill', 'GB'), ('kirill', 'DE'), ('kirill', 'AU');
```

---

Task №2. product2 & country 2 materialized views

```
# The ‘product2’ view should contain the product and its category.
create materialized view product2 as
	select p3.productcategoryid as pcid, p.productid, p3."name" as pcname, p."name" as pname from product p 
	join productsubcategory p2 on p.productsubcategoryid=p2.productsubcategoryid
	join productcategory p3 on p2.productcategoryid=p3.productcategoryid 
with no data;

refresh materialized view product2;

# The ‘country2’ view should be filled with unique codes of the countries where the shops are located (the type of address is Main Office). 

create materialized view country2 as
	select distinct a.countryregioncode as countrycode from customer c
	join customeraddress c1 on c1.customerid = c.customerid
	join address a on c1.addressid = a.addressid
	where c.companyname is not null and c.lastname is null
	and c1.addresstype='Main Office'
with no data;

refresh materialized view country2;

# Allow managers and administrators to read from these views

grant select on product2 to planadmin;
grant select on product2 to planmanager;

grant select on country2 to planadmin;
grant select on country2 to planmanager;

```

---

Task №3. Loading data into the company table

```
insert into company (cname, countrycode, city)
select c.companyname as cname, a.countryregioncode as countrycode, a.city as city
from customer c
join customeraddress c1 on c.customerid = c1.customerid 
join address a on c1.addressid = a.addressid 
where c.companyname is not null and c.lastname is null
and c1.addresstype ='Main Office';
```
---

Task №4. Company classification by annual amount of orders

```
with yearly_sum as (
    select sum(s.subtotal) as yearsum, date_part('y', s.orderdate) as "year" 
    from customer c 
    inner join company c2 on c.companyname = c2.cname
    join salesorderheader s on c.customerid = s.customerid
    where date_part('y', s.orderdate) in (2012, 2013)
    group by "year"),
company_yearly_sales as (
    select c2.id, sum(s.subtotal) as salestotal, date_part('y', s.orderdate) as "year" 
    from customer c 
    inner join company c2 on c.companyname = c2.cname
    join salesorderheader s on c.customerid = s.customerid
    where date_part('y', s.orderdate) in (2012, 2013)
    group by c2.id, "year"
    order by "year" asc, salestotal desc),
company_running_total as (
    select *, sum(salestotal) over(partition by "year" order by salestotal desc) as srt from company_yearly_sales)
insert into company_abc
select id as cid, 
    salestotal, 
    case when srt <= (yearsum * 0.8) then 'A'
    when srt <= (yearsum * 0.95) then 'B'
    else 'C' end as cls,
    t1."year"
from company_running_total as t1
left join yearly_sum as t2 on t1."year"=t2."year";
```
---

Task №5. Finding quarterly sales amount by company, and
product category

```

with company_salesamt as (
	select 
	c.id cid,
	sum(s2.linetotal) salesamt, 
	extract(year from s.orderdate) "year",
	extract (quarter from s.orderdate) quarter_yr,
	p2.pcid categoryid
	from company c 
	join customer c2 on c2.companyname = c.cname
	join salesorderheader s on c2.customerid = s.customerid
	join salesorderdetail s2 on s.salesorderid = s2.salesorderid 
	join product2 p2 on s2.productid = p2.productid
	where date_part('y', s.orderdate) in (2012, 2013)
	group by "year", quarter_yr, c.id, p2.pcid),
cs_v as (
	select cs.*, cs."year" || '.' || cs.quarter_yr as qr, ca.cls as ccls
	from company_salesamt cs
	join company_abc ca on (cs.cid = ca.cid and ca."year"=cs."year"))
insert into company_sales
select cid, salesamt, "year", quarter_yr, qr, categoryid::int, ccls from cs_v;
```

Task №6. Generating the initial planning data

```
import psycopg2
from datetime import datetime

def start_planning(year, quarter, user, pwd):
    """
    plan_status:
        quarterid
        country
        status
        modifieddatetime
        author
    plan_data:
        versionid
        country
        quarterid
        pcid
        salesamt
    """


    queryDeleteFromPlandata = """delete from plan_data where quarterid=%s"""

    queryDeleteFromPlanstatus = """delete from plan_status where quarterid=%s"""

    initialPlanstatus = """insert into plan_status (quarterid, status, country)
    select %s as quarterid, 'R'::text as status, countrycode as country
    from country2"""

    initialPlandata = """with avg_sales_y2 as (
                            select sum(cs.salesamt) s_avg, cs.qr, cs.categoryid, c2.countrycode
                            from company_sales cs
                            join company c2 on cs.cid = c2.id
                            where cs.ccls <> 'C' and cs."year" = %(year2)s and cs.quarter_yr = %(quarter)s
                            group by cs.qr, cs.categoryid, c2.countrycode 
                            order by cs.qr, cs.categoryid, c2.countrycode),
                        avg_sales_y1 as (
                            select sum(cs.salesamt) s_avg, cs.qr, cs.categoryid, c2.countrycode
                            from company_sales cs
                            join company c2 on cs.cid = c2.id
                            where cs.ccls <> 'C' and cs."year" = %(year1)s and cs.quarter_yr = %(quarter)s
                            group by cs.qr, cs.categoryid, c2.countrycode 
                            order by cs.qr, cs.categoryid, c2.countrycode),
                        planed_data as (
                            select 
                            a.countrycode as country, 
                            a.categoryid as pcid,
                            case when a.s_avg is not null and b.s_avg is not null then (a.s_avg + b.s_avg) / 2
                            when a.s_avg is not null then a.s_avg / 2
                            when b.s_avg is not null then b.s_avg / 2
                            else 0 end salesamt
                            from avg_sales_y1 a
                            full outer join avg_sales_y2 b on (a.categoryid=b.categoryid and a.countrycode=b.countrycode)),
                        country_category as (
                            select p2.productcategoryid as pcid, c.countrycode as country
                            from productcategory p2
                            cross join country2 c
                        )
                        insert into plan_data (versionid, country, quarterid, pcid, salesamt)
                        select 
                        'N'::text as versionid, 
                        c_c.country,
                        %(quarterid)s::text as quarterid,
                        c_c.pcid,
                        case when p_d.salesamt is not null then p_d.salesamt else 0 end salesamt
                        from country_category c_c
                        left join planed_data p_d on (c_c.pcid=p_d.pcid and c_c.country=p_d.country);"""

    copyPlandata = """insert into plan_data (versionid, country, quarterid, pcid, salesamt)
                        select 
                        'P' versionid,
                        country,
                        quarterid,
                        pcid,
                        salesamt
                        from plan_data pd
                        where pd.quarterid = %s and versionid ='N' and salesamt > 0;"""

    quarterid = str(year) + '.' + str(quarter)
    
    conn = None

    try:
        conn = psycopg2.connect(database="2020_plans_hanlongzhen", user=user, password=pwd, host="db")

        cur = conn.cursor()

        cur.execute(queryDeleteFromPlandata, [quarterid])

        cur.execute(queryDeleteFromPlanstatus, [quarterid])

        cur.execute(initialPlanstatus, [quarterid])

        cur.execute(initialPlandata, {'quarterid': quarterid, 'quarter': quarter, 'year2': int(year) - 2, 'year1':  int(year) - 1})

        cur.execute(copyPlandata, [quarterid])

        conn.commit()

    except psycopg2.DatabaseError as error:
        conn.rollback()
        print(error)
    finally:
        # close the database communication
        if conn is not None:
            conn.close()

start_planning(2014, 1, 'ivan', '123')
```

Task №7. Changing the plan data

```
def set_lock(year, quarter, user, pwd):

    lockPlanstatus = """update plan_status set status='L', author=current_user, modifieddatetime=current_timestamp 
    where quarterid = %s and status='R' and country in (select country 
	from country_managers
	where username=current_user);
    """

    quarterid = str(year) + '.' + str(quarter)

    conn = None

    try:
        conn = psycopg2.connect(database="2020_plans_hanlongzhen", user=user, password=pwd, host="db")

        cur = conn.cursor()

        cur.execute(lockPlanstatus, [quarterid])

        conn.commit()

    except psycopg2.DatabaseError as error:
        conn.rollback()
        print(error)
    finally:
        # close the database communication
        if conn is not None:
            conn.close()


def remove_lock(year, quarter, user, pwd):

    lockPlanstatus = """update plan_status set status='R', author=current_user, modifieddatetime=current_timestamp 
    where quarterid = %s and status='L' and country in (select country 
	from country_managers
	where username=current_user);
    """

    quarterid = str(year) + '.' + str(quarter)

    conn = None

    try:
        conn = psycopg2.connect(database="2020_plans_hanlongzhen", user=user, password=pwd, host="db")

        cur = conn.cursor()

        cur.execute(lockPlanstatus, [quarterid])

        conn.commit()

    except psycopg2.DatabaseError as error:
        conn.rollback()
        print(error)
    finally:
        # close the database communication
        if conn is not None:
            conn.close()
```

Task №8. Plan data approval

```
def accept_plan(year, quarter, user, pwd):

    deletePlan = """delete from plan_data where quarterid=%s and versionid ='A' and country in 
                    (select country from country_managers
                    where username=current_user);"""

    acceptPlan = """insert into plan_data (versionid, country, quarterid, pcid, salesamt)
                    select 
                    'A' versionid,
                    pd.country,
                    pd.quarterid,
                    pd.pcid,
                    pd.salesamt
                    from plan_data pd
                    inner join plan_status ps on (pd.quarterid=ps.quarterid and pd.country=ps.country)
                    where pd.quarterid = %s and versionid ='P' and ps.status='R' and pd.country in 
                    (select country from country_managers
                    where username=current_user);"""

    updatePlanstatus = """update plan_status set status='A', author=current_user, modifieddatetime=current_timestamp 
                    where quarterid = %s and status='R' and country in (select country 
                    from country_managers
                    where username=current_user);
                    """

    quarterid = str(year) + '.' + str(quarter)

    conn = None

    try:
        conn = psycopg2.connect(database="2020_plans_hanlongzhen", user=user, password=pwd, host="db")

        cur = conn.cursor()

        cur.execute(deletePlan, [quarterid])

        cur.execute(acceptPlan, [quarterid])

        cur.execute(updatePlanstatus, [quarterid])

        conn.commit()

    except psycopg2.DatabaseError as error:
        conn.rollback()
        print(error)
    finally:
        # close the database communication
        if conn is not None:
            conn.close()

accept_plan(2014, 1, 'kirill', '123')
accept_plan(2014, 1, 'sophie', '123')
```

Task №9. Data preparation for plan-fact analysis in Q1 2014

Approcach: Load data of 2014 into the company_sales table and include this table in the view


```
with company_salesamt as (
    select 
    c.id cid,
    sum(s2.linetotal) salesamt, 
    extract(year from s.orderdate) "year",
    extract (quarter from s.orderdate) quarter_yr,
    p2.pcid categoryid
    from company c 
    join customer c2 on c2.companyname = c.cname
    join salesorderheader s on c2.customerid = s.customerid
    join salesorderdetail s2 on s.salesorderid = s2.salesorderid 
    join product2 p2 on s2.productid = p2.productid
    where date_part('y', s.orderdate) in (2014)
    group by "year", quarter_yr, c.id, p2.pcid),
cs_v as (
    select cs.*, cs."year" || '.' || cs.quarter_yr as qr, ca.cls as ccls
    from company_salesamt cs
    join company_abc ca on (cs.cid = ca.cid and ca."year"=2013))
insert into company_sales
select cid, salesamt, "year", quarter_yr, qr, categoryid::int, ccls from cs_v where ccls in ('A', 'B');
```

create view

```
create materialized view mv_plan_fact_2014_q1 as
	with sales_fact as ( 
		select sum(cs.salesamt) salesamt, cs.qr, cs.categoryid, c2.countrycode
		from company_sales cs
		join company c2 on cs.cid = c2.id
		where cs."year" = 2014 and cs.quarter_yr = 1
		group by cs.qr, cs.categoryid, c2.countrycode 
		order by cs.qr, cs.categoryid, c2.countrycode),
	sales_plan as (
		select * from v_plan
	)
	select 
	p.quarterid,
	p.country,
	pc."name",
	(p.salesamt-f.salesamt) as dev,
	(p.salesamt-f.salesamt)/p.salesamt as dev_per
	from sales_fact f 
	join sales_plan p on (f.qr=p.quarterid and f.categoryid=p.pcid and f.countrycode=p.country)
	left join productcategory pc on f.categoryid=pc.productcategoryid
with no data;

refresh materialized view mv_plan_fact_2014_q1;
```


```
create materialized view mv_plan_fact_2014_q1_2 as
	with sales_fact as ( 
		select salesamt, t1."year" || '.' || t1.quarter_yr as qr, categoryid, countrycode from 
		(select 
		sum(s2.linetotal) salesamt, 
		extract(year from s.orderdate) "year",
		extract (quarter from s.orderdate) quarter_yr,
		p2.pcid categoryid,
		c.countrycode 
		from company c 
		join customer c2 on c2.companyname = c.cname
		join salesorderheader s on c2.customerid = s.customerid
		join salesorderdetail s2 on s.salesorderid = s2.salesorderid 
		join product2 p2 on s2.productid = p2.productid
		where date_part('y', s.orderdate) in (2014) and date_part('quarter', s.orderdate) = 1
		group by "year", quarter_yr, p2.pcid, c.countrycode) t1),
	sales_plan as (
		select * from v_plan
	)
	select 
	p.quarterid,
	p.country,
	pc."name",
	(p.salesamt-f.salesamt) as dev,
	(p.salesamt-f.salesamt)/p.salesamt as dev_per
	from sales_fact f 
	join sales_plan p on (f.qr=p.quarterid and f.categoryid=p.pcid and f.countrycode=p.country)
	left join productcategory pc on f.categoryid=pc.productcategoryid
with no data;

refresh materialized view mv_plan_fact_2014_q1_2;
```